# Looker data analysis

This is a data analysis of TheLook's data, a fictitious eCommerce clothing site developed by the Looker team.
 
This is an analysis of TheLooker's sales in order to find insights that to increase the sells the company.
Some insights may help us reaching those selling improvements. For instance, here are some of those:
- ...
- ...
- ...


The datasets used for the analysis are avaible <a href="https://www.kaggle.com/datasets/mustafakeser4/looker-ecommerce-bigquery-dataset">here</a>. 

First, let's import the libraries and each dataset.

In [2]:
import numpy as np
import pandas as pd

events = pd.read_csv('./datasets/events.csv')
inventory_items = pd.read_csv('./datasets/inventory_items.csv')
orders = pd.read_csv('./datasets/orders.csv')
order_items = pd.read_csv('./datasets/order_items.csv')
users = pd.read_csv('./datasets/users.csv')
products = pd.read_csv('./datasets/products.csv')
distribution_centers = pd.read_csv('./datasets/distribution_centers.csv')

## Part 1 : data preprocessing
### 1.1 : data cleaning

We'll try to understand the datasets.
We'll check for missing values:
<ul>
    <li>If there are some missing values we have to investigate whether they were left intentionally empty or if they were unable to fill them. </li>
    <li>We'll also need to fill those missing values with adequate values in order to make accurate analysis and predictions. </li>
</ul>
<br>
For all those operations (preprocessing, exploratory analysis, prediction), the ... will always the
The analysis is centered around the "orders" table as it is the thing that we want to inscrease.   

#### 1.1.1: "orders" data cleaning
Let's have a quick overview of the orders table

In [25]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125226 entries, 0 to 125225
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   order_id      125226 non-null  int64 
 1   user_id       125226 non-null  int64 
 2   status        125226 non-null  object
 3   gender        125226 non-null  object
 4   created_at    125226 non-null  object
 5   returned_at   12530 non-null   object
 6   shipped_at    81461 non-null   object
 7   delivered_at  43884 non-null   object
 8   num_of_item   125226 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 8.6+ MB


In [30]:
orders.head()

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,8,5,Cancelled,F,2022-10-20 10:03:00+00:00,NaN,NaN,NaN,3
1,60,44,Cancelled,F,2023-01-20 02:12:00+00:00,NaN,NaN,NaN,1
2,64,46,Cancelled,F,2021-12-06 09:11:00+00:00,NaN,NaN,NaN,1
3,89,65,Cancelled,F,2020-08-13 09:58:00+00:00,NaN,NaN,NaN,1
4,102,76,Cancelled,F,2023-01-17 08:17:00+00:00,NaN,NaN,NaN,2


Each row of this table represents a summary of the orders a single user took at a point of time:
<ul>
    <li>order_id: the user who made the order<li>
    <li>status: whether this order was cancelled, shipped or delivered<li>
    <li>created_at: date of registration of this order<li>
    <li>shipped_at: date when the order was shipped<li>
    <li>delivered_at: date when the order was delivered<li>
    <li>num_of_items: number of items involved in this order<li>
</ul>

<br>

The datetime fields (returned_at, shipped_at, delivered_at) have missing values

#### 1.1.2 : "events" data cleaning
Let's look at the datatypes of the fields in the "events" dataset

In [27]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2431963 entries, 0 to 2431962
Data columns (total 13 columns):
 #   Column           Dtype  
---  ------           -----  
 0   id               int64  
 1   user_id          float64
 2   sequence_number  int64  
 3   session_id       object 
 4   created_at       object 
 5   ip_address       object 
 6   city             object 
 7   state            object 
 8   postal_code      object 
 9   browser          object 
 10  traffic_source   object 
 11  uri              object 
 12  event_type       object 
dtypes: float64(1), int64(2), object(10)
memory usage: 241.2+ MB


In [29]:
events.head()

,id,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
0,2198523,NaN,3,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:30:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/cancel,cancel
1,1773216,NaN,3,7a3fc3f2-e84f-44fe-8876-eff76741f7a3,2020-08-07 08:41:00+00:00,85.114.141.79,Santa Isabel,São Paulo,07500-000,Safari,Adwords,/cancel,cancel
2,2380515,NaN,3,13d9b2fb-eee1-43fd-965c-267b38dd7125,2021-02-15 18:48:00+00:00,169.250.255.132,Mairiporã,São Paulo,07600-000,IE,Adwords,/cancel,cancel
3,2250597,NaN,3,96f1d44e-9621-463c-954c-d8deb7fffe7f,2022-03-30 10:56:00+00:00,137.25.222.160,Cajamar,São Paulo,07750-000,Chrome,Adwords,/cancel,cancel
4,1834446,NaN,3,d09dce10-a7cb-47d3-a9af-44975566fa03,2019-09-05 01:18:00+00:00,161.114.4.174,São Paulo,São Paulo,09581-680,Chrome,Email,/cancel,cancel


In [5]:
events['traffic_source'].unique()

array(['Adwords', 'Email', 'Facebook', 'YouTube', 'Organic'], dtype=object)

In [6]:
unique_sequence_numbers = events['sequence_number'].unique()
unique_sequence_numbers.sort()
unique_sequence_numbers

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [7]:
events['event_type'].unique()

array(['cancel', 'cart', 'department', 'home', 'product', 'purchase'],
      dtype=object)

At first sights, the fields names are verbose and don't leave a lot of room to ambiguity. Here are what must be the meaning of these fields:
<ul>
<li>id: number/identifier of the event</li>
<li>user_id: identifier of the user that created the event</li>
<li>session_id: session_id of the user's browser</li>
<li>created_at: date when the event occured</li>
<li>ip_address: ip of the hardware used</li>
<li>state: state where the user is</li>
<li>postal_code: postal_code of the place where the user is</li>
<li>browser: browser used</li>
<li>traffic_source: the link/(source) he clicked before getting into the website</li>
<li>uri: website's uri clicked</li>
<li>event_type: what kind of thing did the user triggered? For instance, "cart" means that he </li>
</ul>

For an exploratory analaysis point of view, the "id" field is useless, so let's drop that one.

In [8]:
events.drop(columns=['id'])

,user_id,sequence_number,session_id,created_at,ip_address,city,state,postal_code,browser,traffic_source,uri,event_type
0,NaN,3,83889ed2-2adc-4b9a-af5d-154f6998e778,2021-06-17 17:30:00+00:00,138.143.9.202,São Paulo,São Paulo,02675-031,Chrome,Adwords,/cancel,cancel
1,NaN,3,7a3fc3f2-e84f-44fe-8876-eff76741f7a3,2020-08-07 08:41:00+00:00,85.114.141.79,Santa Isabel,São Paulo,07500-000,Safari,Adwords,/cancel,cancel
2,NaN,3,13d9b2fb-eee1-43fd-965c-267b38dd7125,2021-02-15 18:48:00+00:00,169.250.255.132,Mairiporã,São Paulo,07600-000,IE,Adwords,/cancel,cancel
3,NaN,3,96f1d44e-9621-463c-954c-d8deb7fffe7f,2022-03-30 10:56:00+00:00,137.25.222.160,Cajamar,São Paulo,07750-000,Chrome,Adwords,/cancel,cancel
4,NaN,3,d09dce10-a7cb-47d3-a9af-44975566fa03,2019-09-05 01:18:00+00:00,161.114.4.174,São Paulo,São Paulo,09581-680,Chrome,Email,/cancel,cancel
...,...,...,...,...,...,...,...,...,...,...,...,...
2431958,62906.0,5,12ae51e7-76b3-4560-9335-3ea762eae60e,2022-04-07 01:55:02+00:00,52.92.60.25,St Helens,England,WA9,Firefox,Adwords,/purchase,purchase
2431959,3333.0,5,e472549e-273e-402e-ae5f-38ece25699c2,2023-09-26 10:25:28+00:00,97.78.106.249,Orrell,England,WN5,IE,Email,/purchase,purchase
2431960,42386.0,10,6a7ba027-2af2-46a3-ae05-294a264840cf,2023-07-31 15:11:10+00:00,147.252.137.75,Orrell,England,WN5,Chrome,Facebook,/purchase,purchase
2431961,77546.0,5,29e5960c-214b-456e-8c38-fe6eb8ff1ec8,2024-01-02 14:26:38+00:00,112.209.120.82,Wolverhampton,England,WV6,Chrome,Adwords,/purchase,purchase


Let's check for the null fields on the dataset

In [9]:
count_null_events = events.isnull().sum()
count_null_events

id                       0
user_id            1125671
sequence_number          0
session_id               0
created_at               0
ip_address               0
city                 23080
state                    0
postal_code              0
browser                  0
traffic_source           0
uri                      0
event_type               0
dtype: int64

Well, on a total of 2431963 rows, there are some cells from the "user_id" and "city" columns that are left blank. Were they left blank intentionally or by lack of data? We'll try to answer the question at once.


Concerning the "user_id" column, there are 1,125,671 of the 2,431,963 that are empty, that's almost half of them! (46%).
<br>
If the values on "user_id" were left intentionally empty, it could mean that users weren't connected on those events.
<br>
<br>
Let's admit that the reason why the values of "user_id" are empty is because the users weren't logged on those events. As a result the "user_id" values will never be empty on events that requires to be logged in. In our case, pourchasing is the only event that will always require to be connected in any case. You can't purchase without being logged. That means that it should have a 100% of "user_id" fields are full on those values. , so the "user_id" shouldn't be empty where the event is to "purchase".
<br>
<br>
Let's check the percentage of logged users per event 

In [10]:
events.loc[events['user_id'].notnull(), ['user_id', 'event_type']].groupby(['event_type']).size() * 100 / events.groupby(['event_type']).size()

event_type
cancel               NaN
cart           57.988335
department     58.053695
home          100.000000
product        40.870877
purchase      100.000000
dtype: float64

As we can see, purchase has a connection rate of 100%, which confirms that an event where we have to be connected

In [11]:
events.loc[(events['event_type'] == 'purchase'), ['event_type']].count() # there are 181759 events where users purchased something

event_type    181759
dtype: int64

In [12]:
events.loc[(events['user_id'].notnull()) & (events['event_type'] == 'purchase'), ['user_id']].count() # on the 181759 events, all of the users were connected


user_id    181759
dtype: int64

There are no empty values on the "purchase" events where the users have to be connected.

Now let's dig on why on the some values on the "city" are empty. 

In [13]:
len(events)

2431963

In [14]:
events.loc[events['city'].isnull() & events['user_id'].isnull()].count()

id                 10904
user_id                0
sequence_number    10904
session_id         10904
created_at         10904
ip_address         10904
city                   0
state              10904
postal_code        10904
browser            10904
traffic_source     10904
uri                10904
event_type         10904
dtype: int64

In [15]:
events.loc[events['city'].isnull()]['created_at'].iloc[:100]

358      2021-06-23 12:33:00+00:00
409      2021-12-20 04:32:00+00:00
662      2021-10-21 12:25:00+00:00
889      2021-05-10 18:34:00+00:00
891      2023-07-03 17:55:00+00:00
                   ...            
11186    2021-01-31 22:13:20+00:00
11187    2022-07-24 10:18:01+00:00
11523    2023-11-03 08:31:46+00:00
11524    2023-07-21 14:04:46+00:00
11552    2019-06-06 12:43:00+00:00
Name: created_at, Length: 100, dtype: object

In [16]:
products.head()

,id,cost,category,name,brand,retail_price,department,sku,distribution_center_id
0,13842,2.51875,Accessories,Low Profile Dyed Cotton Twill Cap - Navy W39S55D,MG,6.25,Women,EBD58B8A3F1D72F4206201DA62FB1204,1
1,13928,2.33835,Accessories,Low Profile Dyed Cotton Twill Cap - Putty W39S55D,MG,5.95,Women,2EAC42424D12436BDD6A5B8A88480CC3,1
2,14115,4.87956,Accessories,Enzyme Regular Solid Army Caps-Black W35S45D,MG,10.99,Women,EE364229B2791D1EF9355708EFF0BA34,1
3,14157,4.64877,Accessories,Enzyme Regular Solid Army Caps-Olive W35S45D (...,MG,10.99,Women,00BD13095D06C20B11A2993CA419D16B,1
4,14273,6.50793,Accessories,Washed Canvas Ivy Cap - Black W11S64C,MG,15.99,Women,F531DC20FDE20B7ADF3A73F52B71D0AF,1


In [18]:
inventory_items.head()

,id,product_id,created_at,sold_at,cost,product_category,product_name,product_brand,product_retail_price,product_department,product_sku,product_distribution_center_id
0,67971,13844,2022-07-02 07:09:20+00:00,2022-07-24 06:33:20+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
1,67972,13844,2023-12-20 03:28:00+00:00,NaN,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
2,67973,13844,2023-06-04 02:53:00+00:00,NaN,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
3,72863,13844,2021-10-16 22:58:52+00:00,2021-11-22 02:19:52+00:00,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7
4,72864,13844,2021-08-07 16:33:00+00:00,NaN,2.76804,Accessories,(ONE) 1 Satin Headband,Funny Girl Designs,6.99,Women,2A3E953A5E3D81E67945BCE5519F84C8,7


In [19]:
order_items.head()

,id,order_id,user_id,product_id,inventory_item_id,status,created_at,shipped_at,delivered_at,returned_at,sale_price
0,152013,104663,83582,14235,410368,Cancelled,2023-05-07 06:08:40+00:00,NaN,NaN,NaN,0.02
1,40993,28204,22551,14235,110590,Complete,2023-03-14 03:47:21+00:00,2023-03-15 22:57:00+00:00,2023-03-18 01:08:00+00:00,NaN,0.02
2,51224,35223,28215,14235,138236,Complete,2023-12-05 13:25:30+00:00,2023-12-06 01:20:00+00:00,2023-12-10 10:04:00+00:00,NaN,0.02
3,36717,25278,20165,14235,99072,Shipped,2023-12-22 20:48:19+00:00,2023-12-24 16:44:00+00:00,NaN,NaN,0.02
4,131061,90241,71954,14235,353798,Shipped,2022-06-19 16:57:59+00:00,2022-06-19 19:29:00+00:00,NaN,NaN,0.02
